In [1]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import requests
from langdetect import detect
from bs4 import BeautifulSoup

In [2]:
url = 'https://www.lyrics.com/genres.php?genre='
genres = ['Pop', 'Hip%20Hop','Folk,%20World,%20__%20Country', 'Jazz', 'Rock']

In [3]:
def get_artist(div):
    return div.find_all('p', class_ = "lyric-meta-album-artist")[0].getText()

In [4]:
def get_lyrics_from_url(lyrics_url):
    r = requests.get('https://www.lyrics.com/' + lyrics_url).content
    soup = BeautifulSoup(r, "lxml")
    try : 
        l = soup.find_all('pre')[0].getText()
    except : 
        l = 'NaN'
    return l

In [5]:
def get_title_url(div):
    data = div.find_all('p', class_= "lyric-meta-title")[0]
    lyrics_url = data.find_all('a', href = True)[0]['href']
    title = data.getText()
    return title, lyrics_url

In [6]:
def get_lyrics(lyrics_url):
    l = ''
    while l == '':
        try:
            l = get_lyrics_from_url(lyrics_url)
        except:
            print("Slept for : ", lyrics_url)
            time.sleep(5)
            continue
    return l

In [7]:
def get_artist_from_page(df, genre, nb, url = 'https://www.lyrics.com/genres.php?genre='):
    page_url = url + genre + '&p=' + str(nb)
    resp = requests.get(page_url).content
    soup = BeautifulSoup(resp, "lxml")
    content = soup.find_all('div', class_= "lyric-meta col-sm-6 col-xs-6")
    for c in content:
        t, l = get_title_url(c)
        df.loc[len(df)] = [t, get_artist(c), genre, l]
    return df

In [12]:
genres_emma = genres[:3]
genres_oli = ['Jazz']
MIN_PAGES = 1
MAX_PAGES = 2000 # For Jazz

In [13]:
lyrics_df = pd.DataFrame(columns=['title','artist', 'genre', 'lyrics_url'])

In [15]:
for genre in genres_oli:
    for page in range(MIN_PAGES, MAX_PAGES + 1):
        get_artist_from_page(lyrics_df, genre, page)
        if page % 100 == 0:
            print('finished genre : ', genre, ' page : ', page)

finished genre :  Jazz  page :  100
finished genre :  Jazz  page :  200
finished genre :  Jazz  page :  300
finished genre :  Jazz  page :  400
finished genre :  Jazz  page :  500
finished genre :  Jazz  page :  600
finished genre :  Jazz  page :  700
finished genre :  Jazz  page :  800
finished genre :  Jazz  page :  900
finished genre :  Jazz  page :  1000
finished genre :  Jazz  page :  1100
finished genre :  Jazz  page :  1200
finished genre :  Jazz  page :  1300
finished genre :  Jazz  page :  1400
finished genre :  Jazz  page :  1500
finished genre :  Jazz  page :  1600
finished genre :  Jazz  page :  1700
finished genre :  Jazz  page :  1800
finished genre :  Jazz  page :  1900
finished genre :  Jazz  page :  2000


In [16]:
def clean_df_for_fetch(df):
    print("Initial length of DF: {}".format(len(df)))
    # Remove duplicate songs from same artist
    cleaned_df = lyrics_df.drop_duplicates(['title', 'artist'])
    print("Length of DF without duplications: {}".format(len(cleaned_df)))
    return cleaned_df

lyrics_dropped = clean_df_for_fetch(lyrics_df)

Initial length of DF: 47832
Length of DF without duplications: 10611


In [17]:
lyrics_df['lyrics'] = lyrics_df['lyrics_url'].map(lambda x : get_lyrics_from_url(x))

In [18]:
lyrics_df = lyrics_df.dropna(axis = 1)
lyrics_df = lyrics_df.drop_duplicates('lyrics')
lyrics_df.head(10)

,title,artist,genre,lyrics_url,lyrics
0,Little Things Mean a Lot,Antoinette,Pop,/lyric/31577731/Antoinette/Little+Things+Mean+...,Blow me a kiss from across the room\r\nSay I l...
1,Rhymin' and Stealin',Beastie Boys,Pop,/lyric/2567092/Beastie+Boys/Rhymin%27+and+Stea...,Because mutiny on the bounty's what we're all ...
4,Armada Latina,Cypress Hill,Pop,/lyric/19375506/Cypress+Hill/Armada+Latina,NaN
7,We Want the Funk,Gerardo,Pop,/lyric/1014258/Gerardo/We+Want+the+Funk,"Excuse me, where have I seen you? \r\nThey ten..."
9,Downtown,Kool Moe Dee,Pop,/lyric/32235126/Kool+Moe+Dee/Downtown,"I went to the Moped store, said fuck it\r\nSal..."
10,A Salt With a Deadly Pepa,Salt-N-Pepa,Pop,/lyric/940355/Salt-N-Pepa/A+Salt+With+a+Deadly...,I'm Salt here to resource so start steppin' \r...
11,I Like It Like That,Salt-N-Pepa,Pop,/lyric/940356/Salt-N-Pepa/I+Like+It+Like+That,[You want me to say it now? \r\n\r\n\r\n\r\nc'...
12,Spinderella's Not a Fella (But a Girl D.J.),Salt-N-Pepa,Pop,/lyric/940362/Salt-N-Pepa/Spinderella%27s+Not+...,When the needle's picked up the volume's kicke...
14,Book of Love,The Four Seasons,Pop,/lyric/2598/The+Four+Seasons/Book+of+Love,I write down\r\nEverything that you say\r\nSo ...
15,Love and Other Bruises,Air Supply,Pop,/lyric/3082/Air+Supply/Love+and+Other+Bruises,As the time slips through your fingers\r\nTill...


In [19]:
genre_dict = {'Pop': 'pop', 'Hip%20Hop' : 'hip hop','Folk,%20World,%20__%20Country': 'country', 'Jazz':'jazz', 'Rock':'rock'}

In [111]:
#Avant de quitter le notebook 
"""new_lyrics_df = lyrics_df.drop('lyrics_url', axis = 1)
new_lyrics_df['genre'] = new_lyrics_df['genre'].map(lambda x : genre_dict[x])
new_lyrics_df.to_csv("lyrics_com.csv", index = False)"""

,title,artist,genre,lyrics
0,Little Things Mean a Lot,Antoinette,Pop,Blow me a kiss from across the room\r\nSay I l...
1,Rhymin' and Stealin',Beastie Boys,Pop,Because mutiny on the bounty's what we're all ...
4,Armada Latina,Cypress Hill,Pop,NaN
7,We Want the Funk,Gerardo,Pop,"Excuse me, where have I seen you? \r\nThey ten..."
9,Downtown,Kool Moe Dee,Pop,"I went to the Moped store, said fuck it\r\nSal..."
10,A Salt With a Deadly Pepa,Salt-N-Pepa,Pop,I'm Salt here to resource so start steppin' \r...
11,I Like It Like That,Salt-N-Pepa,Pop,[You want me to say it now? \r\n\r\n\r\n\r\nc'...
12,Spinderella's Not a Fella (But a Girl D.J.),Salt-N-Pepa,Pop,When the needle's picked up the volume's kicke...
14,Book of Love,The Four Seasons,Pop,I write down\r\nEverything that you say\r\nSo ...
15,Love and Other Bruises,Air Supply,Pop,As the time slips through your fingers\r\nTill...


For metal genre, we need to use the styles from the lyrics.com website 

In [8]:
url_style = 'https://www.lyrics.com/styles.php?style='
metal_styles = ['Death+Metal', 'Progressive+Metal', 'Doom+Metal', 'Heavy+Metal', 'Nu+Metal']
MIN_PAGES = 51
MAX_PAGES = 2000

In [9]:
lyrics_df = pd.DataFrame(columns=['title','artist', 'genre', 'lyrics_url'])

In [10]:
for m_style in metal_styles:
    for page in range(MIN_PAGES, MAX_PAGES + 1):
        length_df = len(lyrics_df)
        get_artist_from_page(lyrics_df, m_style, page, url = url_style)
        # If we go above the real page number, the dataframe will have the same size (no more lyrics)
        if length_df == len(lyrics_df):
            print("No more pages for {}".format(m_style))
            break
        print('finished genre : ', m_style, ' page : ', page)

finished genre :  Death+Metal  page :  51
finished genre :  Death+Metal  page :  52
finished genre :  Death+Metal  page :  53
finished genre :  Death+Metal  page :  54
finished genre :  Death+Metal  page :  55
finished genre :  Death+Metal  page :  56
finished genre :  Death+Metal  page :  57
finished genre :  Death+Metal  page :  58
finished genre :  Death+Metal  page :  59
finished genre :  Death+Metal  page :  60
finished genre :  Death+Metal  page :  61
finished genre :  Death+Metal  page :  62
finished genre :  Death+Metal  page :  63
finished genre :  Death+Metal  page :  64
finished genre :  Death+Metal  page :  65
finished genre :  Death+Metal  page :  66
finished genre :  Death+Metal  page :  67
finished genre :  Death+Metal  page :  68
finished genre :  Death+Metal  page :  69
finished genre :  Death+Metal  page :  70
finished genre :  Death+Metal  page :  71
finished genre :  Death+Metal  page :  72
finished genre :  Death+Metal  page :  73
finished genre :  Death+Metal  pag

finished genre :  Heavy+Metal  page :  202
finished genre :  Heavy+Metal  page :  203
finished genre :  Heavy+Metal  page :  204
finished genre :  Heavy+Metal  page :  205
finished genre :  Heavy+Metal  page :  206
finished genre :  Heavy+Metal  page :  207
finished genre :  Heavy+Metal  page :  208
finished genre :  Heavy+Metal  page :  209
finished genre :  Heavy+Metal  page :  210
finished genre :  Heavy+Metal  page :  211
finished genre :  Heavy+Metal  page :  212
finished genre :  Heavy+Metal  page :  213
finished genre :  Heavy+Metal  page :  214
finished genre :  Heavy+Metal  page :  215
finished genre :  Heavy+Metal  page :  216
finished genre :  Heavy+Metal  page :  217
finished genre :  Heavy+Metal  page :  218
finished genre :  Heavy+Metal  page :  219
finished genre :  Heavy+Metal  page :  220
finished genre :  Heavy+Metal  page :  221
finished genre :  Heavy+Metal  page :  222
finished genre :  Heavy+Metal  page :  223
finished genre :  Heavy+Metal  page :  224
finished ge

finished genre :  Heavy+Metal  page :  393
finished genre :  Heavy+Metal  page :  394
finished genre :  Heavy+Metal  page :  395
finished genre :  Heavy+Metal  page :  396
finished genre :  Heavy+Metal  page :  397
finished genre :  Heavy+Metal  page :  398
finished genre :  Heavy+Metal  page :  399
finished genre :  Heavy+Metal  page :  400
finished genre :  Heavy+Metal  page :  401
finished genre :  Heavy+Metal  page :  402
finished genre :  Heavy+Metal  page :  403
finished genre :  Heavy+Metal  page :  404
finished genre :  Heavy+Metal  page :  405
finished genre :  Heavy+Metal  page :  406
finished genre :  Heavy+Metal  page :  407
finished genre :  Heavy+Metal  page :  408
finished genre :  Heavy+Metal  page :  409
finished genre :  Heavy+Metal  page :  410
finished genre :  Heavy+Metal  page :  411
finished genre :  Heavy+Metal  page :  412
finished genre :  Heavy+Metal  page :  413
finished genre :  Heavy+Metal  page :  414
finished genre :  Heavy+Metal  page :  415
finished ge

finished genre :  Heavy+Metal  page :  584
finished genre :  Heavy+Metal  page :  585
finished genre :  Heavy+Metal  page :  586
finished genre :  Heavy+Metal  page :  587
finished genre :  Heavy+Metal  page :  588
finished genre :  Heavy+Metal  page :  589
finished genre :  Heavy+Metal  page :  590
finished genre :  Heavy+Metal  page :  591
finished genre :  Heavy+Metal  page :  592
finished genre :  Heavy+Metal  page :  593
finished genre :  Heavy+Metal  page :  594
finished genre :  Heavy+Metal  page :  595
finished genre :  Heavy+Metal  page :  596
finished genre :  Heavy+Metal  page :  597
finished genre :  Heavy+Metal  page :  598
finished genre :  Heavy+Metal  page :  599
finished genre :  Heavy+Metal  page :  600
finished genre :  Heavy+Metal  page :  601
finished genre :  Heavy+Metal  page :  602
finished genre :  Heavy+Metal  page :  603
finished genre :  Heavy+Metal  page :  604
finished genre :  Heavy+Metal  page :  605
finished genre :  Heavy+Metal  page :  606
finished ge

In [54]:
print(len(lyrics_df))
lyrics_dropped = lyrics_df.drop_duplicates(['title', 'artist'])
print(len(lyrics_dropped))

4595
2380


In [17]:
#lyrics = lyrics_df['lyrics_url'].map(lambda x : get_lyrics(x))
lyrics_list = []
for i, row in lyrics_dropped.iterrows():
    if i % 100 == 99:
        print("Completion: {}".format(len(lyrics_list) / len(lyrics_dropped)))
    url = row['lyrics_url']
    lyrics = get_lyrics(url)
    lyrics_list.append(lyrics)

Completion: 0.006219960418433701
Completion: 0.015078691923475638
Completion: 0.023937423428517575
Completion: 0.032796154933559515
Completion: 0.04165488643860145
Completion: 0.05814720572990293
Completion: 0.06625200263877108
Completion: 0.07473376684572614
Completion: 0.0824615964565074
Completion: 0.09047215154085383
Completion: 0.0978230138535482
Completion: 0.10922627462067666
Completion: 0.11506926774102347
Completion: 0.12176043728206579
Completion: 0.1284516068231081
Completion: 0.13542550183771557
Completion: 0.141739704080671
Completion: 0.14729997172745266
Completion: 0.1594571670907549
Completion: 0.16671378757892752
Completion: 0.1799076430119687
Completion: 0.1856563943077938
Completion: 0.20780322307039864
Completion: 0.21496560173404958
Completion: 0.22702855527283008
Completion: 0.23164640467439448
Completion: 0.24795024031665253
Completion: 0.2533220243143907
Completion: 0.2586938083121289
Completion: 0.2639713504853454
Completion: 0.2692488926585619
Completion: 0.27

In [57]:
print(len(lyrics_list))
lyrics_dropped['lyrics'] = lyrics_list
#lyrics_df = lyrics_df.dropna(axis = 1)
#lyrics_df = lyrics_df.drop_duplicates('lyrics')
lyrics_dropped.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,title,artist,genre,lyrics_url,lyrics
0,Transmigration,Crematory,Death+Metal,/lyric/32932/Crematory/Transmigration,The end of your time has come\r\nA journey int...
1,Reek of Putrefaction,Carcass,Death+Metal,/lyric/32964/Carcass/Reek+of+Putrefaction,NaN
2,The Law,Exhorder,Death+Metal,/lyric/44038/Exhorder/The+Law,Do what thou will shall\r\nbe whole of the law...
3,Dead Embryonic Cells,Sepultura,Death+Metal,/lyric/294633/Sepultura/Dead+Embryonic+Cells,Land of anger\r\nI didn't ask to be born\r\nSa...
4,Lost Equilibrium,Convulse,Death+Metal,/lyric/310185/Convulse/Lost+Equilibrium,NaN


In [68]:
lyrics_dropped2 = lyrics_dropped.drop_duplicates('lyrics')
lyrics_dropped2 = test.dropna(subset = ['lyrics'])
len(lyrics_dropped2)

1559

In [71]:
# Check if english language
def language_of_lyrics(lyrics):
    try:
        lang = detect(lyrics)
        return lang
    except:
        return "no lang"

def get_list_langs(df):
    list_langs = []
    for i, row in df.iterrows():
        if i % 1000 == 999:
            print("Completion: {}".format(len(list_langs) / len(df)))
        lang = language_of_lyrics(row['lyrics'])
        list_langs.append(lang)
    return list_langs

langs = get_list_langs(lyrics_dropped2)

Completion: 0.18473380372033354
Completion: 0.6703014753046825
Completion: 0.9018601667735728


In [74]:
lyrics_dropped2['lang'] = langs
new_lyrics_df = lyrics_dropped2[lyrics_dropped2['lang'] == 'en']
print ("Length of final dataframe: {}".format(len(new_lyrics_df)))

genres = ['metal'] * len(new_lyrics_df)
new_lyrics_df['genre'] = genres
new_lyrics_df = new_lyrics_df.drop('lyrics_url', axis = 1)
new_lyrics_df.to_csv("../kaggleDataset/lyrics_metal.csv", index = False)

Length of final dataframe: 1523


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
